### 6.2. Parameter Management
 
Once we have chosen an architecture and set our hyperparameters, we proceed to the training loop, where our goal is to find parameter values that minimize our loss function. After training, we will need these parameters in order to make future predictions. Additionally, we will sometimes wish to extract the parameters perhaps to reuse them in some other context, to save our model to disk so that it may be executed in other software, or for examination in the hope of gaining scientific understanding.

Most of the time, we will be able to ignore the nitty-gritty details of how parameters are declared and manipulated, relying on deep learning frameworks to do the heavy lifting. However, when we move away from stacked architectures with standard layers, we will sometimes need to get into the weeds of declaring and manipulating parameters. In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualizations.

* Sharing parameters across different model com

In [3]:
import tensorflow as tf

In [4]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X).shape

TensorShape([2, 1])

In [27]:
net.layers

In [29]:
for layer in net.layers:
  print(f"Layer name: {layer.name}, Output shape: {layer.output_shape}")

Layer name: flatten_2, Output shape: (2, 4)
Layer name: dense_4, Output shape: (2, 4)
Layer name: dense_5, Output shape: (2, 1)


In [36]:
print(tf.__version__)

2.15.0


#### 6.2.1. Parameter Access

Let’s start with how to access parameters from the models that you already know.
When a model is defined via the Sequential class, we can first access any layer by indexing into the model as though it were a list. Each layer’s parameters are conveniently located in its attribute.
We can inspect the parameters of the second fully connected layer as follows.

In [2]:
net.layers[2].weights

[<tf.Variable 'dense_1/kernel:0' shape=(4, 1) dtype=float32, numpy=
 array([[-0.78668416],
        [ 0.4327531 ],
        [-0.40664268],
        [-0.28720444]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

We can see that this fully connected layer contains two parameters, corresponding to that layer’s weights and biases, respectively.

#### 6.2.1.1. Targeted Parameters

Note that each parameter is represented as an instance of the parameter class. To do anything useful with the parameters, we first need to access the underlying numerical values. There are several ways to do this. Some are simpler while others are more general. The following code extracts the bias from the second neural network layer, which returns a parameter class instance, and further accesses that parameter’s value.

In [6]:
type(net.layers[2].weights[1]), tf.convert_to_tensor(net.layers[2].weights[1])

(tensorflow.python.ops.resource_variable_ops.ResourceVariable,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>)

#### 6.2.1.2. All Parameters at Once
When we need to perform operations on all parameters, accessing them one-by-one can grow tedious. The situation can grow especially unwieldy when we work with more complex, e.g., nested, modules, since we would need to recurse through the entire tree to extract each sub-module’s parameters. Below we demonstrate accessing the parameters of all layers.

In [25]:
net.layers,   net.get_weights()

([<keras.src.layers.reshaping.flatten.Flatten at 0x7fc9e19a2e90>,
 [array([[ 0.03152835, -0.20600021, -0.11119342,  0.13039881],
         [ 0.8349889 , -0.36418748, -0.58995926,  0.5881589 ],
         [ 0.23226136,  0.49354047, -0.5157879 ,  0.10499209],
         [-0.43608382, -0.23599237, -0.5114808 ,  0.36620098]],
        dtype=float32),
  array([0., 0., 0., 0.], dtype=float32),
  array([[-0.23940843],
         [ 0.7190784 ],
         [ 0.5216007 ],
         [-0.35595208]], dtype=float32),
  array([0.], dtype=float32)])

#### 6.2.2. Tied Parameters
Often, we want to share parameters across multiple layers. Let’s see how to do this elegantly. In the following we allocate a fully connected layer and then use its parameters specifically to set those of another layer. Here we need to run the forward propagation net(X) before accessing the parameters.

In [10]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


### 6.3. Parameter Initialization
 
Now that we know how to access the parameters, let’s look at how to initialize them properly. We discussed the need for proper initialization in Section 5.4. The deep learning framework provides default random initializations to its layers. However, we often want to initialize our weights according to various other protocols. The framework provides most commonly used protocols, and also allows to create a custom initializer.

By default, Keras initializes weight matrices uniformly by drawing from a range that is computed according to the input and output dimension, and the bias parameters are all set to zero. TensorFlow provides a variety of initialization methods both in the root module and the keras.initializers module.

In [38]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X).shape

TensorShape([2, 1])

#### 6.3.1. Built-in Initialization
Let’s begin by calling on built-in initializers. The code below initializes all weight parameters as Gaussian random variables with standard deviation 0.01, while bias parameters are cleared to zero.

In [41]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X) # weights get created with the instanceiation. else error
net.weights[0], net.weights[1]

(<tf.Variable 'dense_12/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.0025312 , -0.02097435,  0.01360615, -0.00122448],
        [-0.02847254, -0.00691849,  0.01073523,  0.00374302],
        [ 0.00736204,  0.00382908,  0.00891054, -0.01125475],
        [ 0.00410018, -0.00641336, -0.0057174 , -0.00123267]],
       dtype=float32)>,
 <tf.Variable 'dense_12/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

We can also initialize all the parameters to a given constant value (say, 1).

In [42]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_14/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_14/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

We can also apply different initializers for certain blocks. For example, below we initialize the first layer with the Xavier initializer and initialize the second layer to a constant value of 42.

In [47]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5 ,activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_24/kernel:0' shape=(4, 5) dtype=float32, numpy=
array([[-0.13218945,  0.75522876, -0.75359315,  0.15835404,  0.7877854 ],
       [ 0.6005722 , -0.02939814,  0.5569992 , -0.7584791 ,  0.63693726],
       [ 0.46216822, -0.6875799 ,  0.8145089 ,  0.5875597 , -0.31115454],
       [-0.58594376, -0.29458034,  0.4479493 ,  0.32112324,  0.05051011]],
      dtype=float32)>
<tf.Variable 'dense_25/kernel:0' shape=(5, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>
